# LangGraph Chatbot - How It Works

This notebook explains the working of a conversational AI agent built with LangGraph and Ollama.

## 1. Import Dependencies

We import the necessary components:
- **StateGraph**: Core component for building the conversation flow
- **Message types**: For structuring conversation data
- **ChatOllama**: Interface to the local Ollama model
- **TypedDict & Annotations**: For type safety and state management

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_community.chat_models import ChatOllama
from typing import TypedDict, Annotated, Sequence
from operator import add

## 2. Define Agent State

The `AgentState` is the data structure that flows through our graph. It contains:
- **messages**: A sequence of conversation messages
- **Annotated with `add`**: This allows new messages to be appended to existing ones
- **Type hints**: Ensures only HumanMessage or AIMessage objects are stored

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[HumanMessage | AIMessage], add]

## 3. Initialize Language Model and System Prompt

- **ChatOllama**: Creates connection to the local Gemma 2B model
- **Temperature 0.7**: Controls randomness (0=deterministic, 1=very random)
- **SystemMessage**: Sets the AI's personality and behavior guidelines

In [ ]:
llm_model = ChatOllama(model="gemma:2b", temperature=0.7)

sys_prompt = SystemMessage(
    content="You are a helpful, general-purpose AI assistant. Answer clearly and politely."
)

## 4. Define the LLM Node Function

This is the core processing function that:
1. Takes the current state (conversation history)
2. Prepends the system prompt to guide behavior
3. Sends all messages to the language model
4. Returns the AI's response in the correct state format

The function signature matches LangGraph's node requirements.

In [ ]:
def llm_node(state: AgentState) -> AgentState:
    messages = [sys_prompt] + list(state["messages"])
    response = llm_model.invoke(messages)
    return {"messages": [response]}

## 5. Create the Conversation Graph

The graph defines the flow of conversation:
1. **StateGraph(AgentState)**: Creates a graph that manages our state type
2. **add_node**: Registers our LLM function as a processing node
3. **set_entry_point**: Defines where conversations start
4. **add_edge**: Creates a path from LLM to END (conversation ends after each response)
5. **compile()**: Optimizes the graph for execution

This creates a simple linear flow: Input → LLM Processing → Output

In [ ]:
def create_agent():
    graph = StateGraph(AgentState)
    graph.add_node("llm_model", llm_node)
    graph.set_entry_point("llm_model")
    graph.add_edge("llm_model", END)
    return graph.compile()

bot = create_agent()

## 6. Interactive Chat Interface

This function provides a command-line chat interface:
- **Conversation loop**: Continuously accepts user input
- **History management**: Maintains context across exchanges
- **State updates**: Each message is added to history for context
- **Error handling**: Gracefully handles interruptions and errors

The bot maintains memory by keeping all previous messages in the history list.

In [ ]:
def run_bot():
    print("Agent007 Bot - Ready to assist!")
    print("Type 'exit' or 'bye' to quit the conversation.")
    print("-" * 50)
    
    history = []
    while True:
        try:
            user_input = input("\nYou: ").strip()
            
            if user_input.lower() in ["exit", "bye", "quit"]:
                print("\n007: Goodbye! Have a great day!")
                break
            
            if not user_input:
                print("Please enter a message or type 'exit' to quit.")
                continue
                
            user_msg = HumanMessage(content=user_input)
            history.append(user_msg)
            
            result = bot.invoke({"messages": history})
            ai_msg = result["messages"][-1]
            
            print(f"\n007: {ai_msg.content}")
            history.append(ai_msg)
            
        except KeyboardInterrupt:
            print("\n\n007: Conversation interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"Error: {e}")
            print("Please try again or type 'exit' to quit.")

## 7. Single Question Function

This utility function allows asking single questions:
- **Optional history**: Can include previous conversation context
- **Message wrapping**: Converts string input to proper message format
- **Direct response**: Returns just the text content, not the full message object

Useful for programmatic interactions or one-off questions.

In [ ]:
def ask_bot(question: str, conversation_history=None):
    if conversation_history is None:
        conversation_history = []
    
    user_msg = HumanMessage(content=question)
    messages = conversation_history + [user_msg]
    
    result = bot.invoke({"messages": messages})
    return result["messages"][-1].content

## 8. How the Data Flows

Here's what happens when you send a message:

1. **User Input** → Converted to HumanMessage
2. **History Update** → Message added to conversation history
3. **State Creation** → History wrapped in AgentState format
4. **Graph Execution** → bot.invoke() runs the graph
5. **LLM Processing** → llm_node() processes the state
6. **Model Call** → System prompt + history sent to Gemma
7. **Response Generation** → Model generates AIMessage response
8. **State Return** → New message returned in state format
9. **History Update** → AI response added to conversation history
10. **Display** → Response shown to user

## 9. Demonstration

Let's see the bot in action with a simple example:

In [ ]:
response = ask_bot("Hello! What can you help me with?")
print(f"Bot: {response}")

print("\n" + "="*50)
print("To start interactive chat, run: run_bot()")
print("To ask single questions, use: ask_bot('your question')")

## 10. Key Architecture Benefits

**LangGraph provides several advantages:**

- **State Management**: Automatic handling of conversation context
- **Modular Design**: Easy to add new nodes (tools, memory, etc.)
- **Type Safety**: Ensures data consistency across the graph
- **Scalability**: Can handle complex multi-step reasoning
- **Debugging**: Clear visibility into each step of processing

**This simple example demonstrates:**
- Linear conversation flow
- Context preservation across turns
- Integration with local LLMs
- Clean separation of concerns

## 11. Usage Examples

Try these in the next cell:

In [ ]:
# run_bot()

# ask_bot("Explain quantum computing in simple terms")

# history = []
# response1 = ask_bot("My name is Alice", history)
# history.extend([HumanMessage(content="My name is Alice"), AIMessage(content=response1)])
# response2 = ask_bot("What's my name?", history)
# print(response2)